## Univariate and Multivariate Modeling and best fit - BASEBALL

### Create univariate linear regression to predict player salary

In [1]:
import pandas
import numpy as np
# from scipy.stats import 
import matplotlib.pyplot as plt

PLAYER_SALARY = './baseball_data/Salaries.csv'
salary = pandas.read_csv(PLAYER_SALARY)
salary.head(4)

SyntaxError: invalid syntax (<ipython-input-1-72e0a11dc920>, line 3)